In [1]:
#!/usr/bin/env python

import socket
import time
from datetime import datetime
import threading
import pyvisa

rm = pyvisa.ResourceManager()
rm.list_resources('?*')
inst = rm.open_resource('USB0::1027::60786::100031819913::0::RAW')
inst.write('*RST')


6

In [5]:
from thorlabs_mc2000b import MC2000B, Blade
import serial
from serial.tools import list_ports, list_ports_common
import re

def find_device(serial_number=""):
    """
    Search attached serial ports for a Thorlabs MC2000B.

    The first device found will be returned.
    If multiple choppers are attached to the system, the ``serial_number`` parameter may be used
    to select the correct device. This is a regular expression match, for example
    ``serial_number="21"`` would match devices with serial numbers starting with 21, while
    ``serial_number=".*21$"`` would match devices ending in 21.

    :param serial_number: Regular expression to match a device serial number.
    """
    for p in serial.tools.list_ports.comports():
        # If manufacturer and product fields exist, try to use them
        # We require a match on serial number though
        print(p.manufacturer,p.product)
        if (p.manufacturer in ("Thorlabs", "FTDI")
            and (p.product == "MC2000 Optical Chopper" if p.product else True)
            and (re.match(serial_number, p.serial_number) if p.serial_number else False)):
            print('found ->',p)
            return p
        
chopper = MC2000B(serial_port = find_device())

Thorlabs MC2000 Optical Chopper
found -> /dev/ttyUSB2 - MC2000 Optical Chopper - MC2000 Optical Chopper


In [3]:
# pip install pyvisa, pyvisa-py
# sudo echo 'SUBSYSTEM=="usb", MODE="0666", GROUP="usbusers"' >> /etc/udev/rules.d/99-com.rules
# sudo usermod -a -G tty trygvrad
def query(query):
    if 'chopper' in query:
        try:
            out = getattr(chopper, query.split(':')[1].strip('?'))
        except:
            out = f'unable to find {query}'
    else:
        inst.write(query)
        out =''
        while True:
            line = inst.read_bytes(1)
            #print(line)
            if line == b'1`':
                if len(out)>0:
                    break
                continue
            out += str(line)[4:-3]
            if b'\n' in line:
                break
    return out
print('initial query',query('*ESR?'))
inst.write('FUNC PULS')
#FUNCtion {SINusoid | SQUare | RAMP | PULSe | ARBitrary}
query('FUNC?')

HMF2525_keys = [
    'FUNC',
    'OUTPut',
    'FREQuency',
    'PERiod',
    'VOLTage',
    'VOLTage:UNIT',
    'VOLTage:HIGH',
    'VOLTage:LOW',
    'VOLTage:OFFSet',
    'FUNCtion:PULSe:WIDTh:HIGH',
    'FUNCtion:PULSe:WIDTh:LOW',
    'FUNCtion:PULSe:DCYCle',
    'FUNCtion:PULSe:ETIMe',
    'TRIGger:SOURce',# {IMMediate | EXTernal}
    
    'BURSt:MODE',# TRIGgered | GATed
    'BURSt:NCYCles',
    'BURSt:INTernal:PERiod',
    'BURSt:PHASe', #0 to 360
    'BURSt:STATe',
    ]
def get_params():
    parameters = {}
    for key in HMF2525_keys:
        print(f'    HMF2525 query {key}?')
        parameters[key] = query(key+'?')
    return parameters

def set_params(parameters):
    for key in parameters.keys():
        time.sleep(0.1)
        inst.write(key+' '+str(parameters[key]))

def parse_single_param(key, val):
    #time.sleep(0.1)
    inst.write(key+' '+str(val))
    print(f'   set HMF2525  {key}: {val}')
    
def parse_parameters(parameters):
    for key in parameters.keys():
        parse_single_param(key, parameters[key])

initial query 0


In [ ]:
def data_to_dict(inp):
    spl = inp.split("', '")
    spl[0] = spl[0][2:]
    spl[-1] = spl[-1][:-2]
    parameters = {}
    for i, sp in enumerate(spl):
        key_val = sp.split("': '")
        parameters[key_val[0]] = key_val[1]
    return parameters




class client(threading.Thread):
    def __init__(self, socket, address):
        threading.Thread.__init__(self)
        self.sock = socket
        self.addr = address
        self.start()

    def run(self):
        data = self.sock.recv(BUFFER_SIZE).decode()
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Current Time =", current_time)
        if data == '?':
            parameters = get_params()
            self.sock.send(str(parameters).encode())
            print('    sent parameters')
        elif '=' in data:
            print('    recieved single val')
            key = data.split('=')[0].strip()
            val = data.split('=')[1].strip()
            parse_single_param(key, val)
            self.reply_single(key)
        elif data[0] == '?' or data[-1]=='?':
            print('    recieved single query')
            self.reply_single(data.strip('?'))
        else:
            parameters = data_to_dict(data)
            parse_parameters(parameters)
            #process(parameters)
            #conn.close()
    def reply_single(self, key):
            val = query(key+'?')
            self.sock.send(str(key+' = '+ str(val)).encode())

'''def run(TCP_IP = '127.0.0.1', TCP_PORT = 5005, BUFFER_SIZE = 1024):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((TCP_IP, TCP_PORT))
    s.listen(1)

    while 1:
        conn, addr = s.accept()
        #print('Connection address:', addr)
        data = conn.recv(BUFFER_SIZE).decode()
        
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Current Time =", current_time)
        if data == '?':
            parameters = get_params()
            s.send(str(parameters).encode())
            print('    sent parameters')
        else:
            parameters = data_to_dict(data)
            parse_parameters(parameters)
            #process(parameters)
            #conn.close()
run()'''

serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = "127.0.0.1"
port = 5006
BUFFER_SIZE = 1024
serversocket.bind((host, port))
serversocket.listen(5)
print ('server started and listening')
while 1:
    clientsocket, address = serversocket.accept()
    client(clientsocket, address)

server started and listening
Current Time = 21:12:21
    HMF2525 query FUNC?
    HMF2525 query OUTPut?
    HMF2525 query FREQuency?
    HMF2525 query PERiod?
    HMF2525 query VOLTage?
    HMF2525 query VOLTage:UNIT?
    HMF2525 query VOLTage:HIGH?
    HMF2525 query VOLTage:LOW?
    HMF2525 query VOLTage:OFFSet?
    HMF2525 query FUNCtion:PULSe:WIDTh:HIGH?
    HMF2525 query FUNCtion:PULSe:WIDTh:LOW?
    HMF2525 query FUNCtion:PULSe:DCYCle?
    HMF2525 query FUNCtion:PULSe:ETIMe?
    HMF2525 query TRIGger:SOURce?
    HMF2525 query BURSt:MODE?
    HMF2525 query BURSt:NCYCles?
    HMF2525 query BURSt:INTernal:PERiod?
    HMF2525 query BURSt:PHASe?
    HMF2525 query BURSt:STATe?
    sent parameters
Current Time = 21:12:25
    recieved single val
   set chopper  phase: 180
Current Time = 21:12:27
    recieved single val
ERROR: don't know wha to do with OUTput: ON, skipping
Current Time = 21:12:32
    recieved single val
ERROR: don't know wha to do with OUTput: ON, skipping
Current Time = 21